# Obtaining the Data

This Jupyter Notebook runs through the process of obtaining the data of the 45 M&A transactions that are studied in the Dissertation. 
The structure is the following:

1. Obtain the List of 45 M&A transactions
2. Obtain and clean the data for the Banks in the sample
3. Create the matrices for the DSBM and SBM Model.

In [244]:
import numpy as np
import pandas as pd

## 1. Obtaining the List of the 45 M&A transactions

First a list of all M&A activitiesbetween 2014 and 2017 needs to be obtained:

In [245]:
MATrans1417 = pd.read_csv("Data/MAActivities2014-2017.csv") # List of all MA transactions in the observational period
MATrans1219 = pd.read_csv("Data/MAActivities2012-2019.csv",on_bad_lines="skip") # List of all MA transactions in the +-2 year period

MATrans1417 = MATrans1417[["ACQ_CERT","ACQ_INSTNAME","OUT_CERT","OUT_INSTNAME","EFFYEAR"]]
MATrans1219 = MATrans1219[["ACQ_CERT","ACQ_INSTNAME","OUT_CERT","OUT_INSTNAME","EFFYEAR"]]

The following function matches each M&A transaction to the corresponding data. Here the combined assets of the previous year and the combined deposits of the previous year are used. Note that only the former will be used later on. The data for this is obtained from the FDIC.

In [246]:
def getMAAssetsAndDeposits(MATrans1417,deflator):
    '''This helper function returns the summary statistics for choosing M&A Deals according to the Dissertation'''
    EasyData2013 = pd.read_csv("Data/Financials_L1_formatted_2013.csv")
    EasyData2014 = pd.read_csv("Data/Financials_L1_formatted_2014.csv")
    EasyData2015 = pd.read_csv("Data/Financials_L1_formatted_2015.csv")
    EasyData2016 = pd.read_csv("Data/Financials_L1_formatted_2016.csv")

    # Number of MA Deals
    nDeals = MATrans1417.shape[0]


    # Empty Vector of the combined Assets/Deposits of the Previous Year
    AssetVec = np.zeros(shape = nDeals)
    DepositVec = np.zeros(shape = nDeals)

    for i in range(nDeals):

        # ID for Target and Acquirer
        AcqID = MATrans1417["ACQ_CERT"][i]
        TarID = MATrans1417["OUT_CERT"][i]
        Year = MATrans1417["EFFYEAR"][i]

        # Depending on Year the Data of the previous Year is obtained, added and deflated       
        if Year == 2014:
            AcqAsset = EasyData2013[EasyData2013["Cert/ID"] == AcqID]["Total Assets"].to_numpy()
            TarAsset = EasyData2013[EasyData2013["Cert/ID"] == TarID]["Total Assets"].to_numpy()

            if len(AcqAsset) == 0  or len(TarAsset) == 0:
                AssetVec[i] = None
            else:
                AssetVec[i] = (AcqAsset + TarAsset)/deflator[Year]

            AcqDep = EasyData2013[EasyData2013["Cert/ID"] == AcqID]["Total Domestic Deposits"].to_numpy()
            TarDep = EasyData2013[EasyData2013["Cert/ID"] == TarID]["Total Domestic Deposits"].to_numpy()

            if len(AcqDep) == 0  or len(TarDep) == 0:
                DepositVec[i] = None
            else:
                DepositVec[i] = (AcqDep + TarDep)/deflator[Year]

        if Year == 2015:
            AcqAsset = EasyData2014[EasyData2014["Cert/ID"] == AcqID]["Total Assets"].to_numpy()
            TarAsset = EasyData2014[EasyData2014["Cert/ID"] == TarID]["Total Assets"].to_numpy()

            if len(AcqAsset) == 0  or len(TarAsset) == 0:
                AssetVec[i] = None
            else:
                AssetVec[i] = (AcqAsset + TarAsset)/deflator[Year]

            AcqDep = EasyData2014[EasyData2014["Cert/ID"] == AcqID]["Total Domestic Deposits"].to_numpy()
            TarDep = EasyData2014[EasyData2014["Cert/ID"] == TarID]["Total Domestic Deposits"].to_numpy()

            if len(AcqDep) == 0  or len(TarDep) == 0:
                DepositVec[i] = None
            else:
                DepositVec[i] = (AcqDep + TarDep)/deflator[Year]

        if Year == 2016:
            AcqAsset = EasyData2015[EasyData2015["Cert/ID"] == AcqID]["Total Assets"].to_numpy()
            TarAsset = EasyData2015[EasyData2015["Cert/ID"] == TarID]["Total Assets"].to_numpy()

            if len(AcqAsset) == 0  or len(TarAsset) == 0:
                AssetVec[i] = None
            else:
                AssetVec[i] = (AcqAsset + TarAsset)/deflator[Year]

            AcqDep = EasyData2015[EasyData2015["Cert/ID"] == AcqID]["Total Domestic Deposits"].to_numpy()
            TarDep = EasyData2015[EasyData2015["Cert/ID"] == TarID]["Total Domestic Deposits"].to_numpy()

            if len(AcqDep) == 0  or len(TarDep) == 0:
                DepositVec[i] = None
            else:
                DepositVec[i] = (AcqDep + TarDep)/deflator[Year]

        if Year == 2017:
            AcqAsset = EasyData2016[EasyData2016["Cert/ID"] == AcqID]["Total Assets"].to_numpy()
            TarAsset = EasyData2016[EasyData2016["Cert/ID"] == TarID]["Total Assets"].to_numpy()

            if len(AcqAsset) == 0  or len(TarAsset) == 0:
                AssetVec[i] = None
            else:
                AssetVec[i] = (AcqAsset + TarAsset)/deflator[Year]

            AcqDep = EasyData2016[EasyData2016["Cert/ID"] == AcqID]["Total Domestic Deposits"].to_numpy()
            TarDep = EasyData2016[EasyData2016["Cert/ID"] == TarID]["Total Domestic Deposits"].to_numpy()

            if len(AcqDep) == 0  or len(TarDep) == 0:
                DepositVec[i] = None
            else:
                DepositVec[i] = (AcqDep + TarDep)/deflator[Year]


        
        

    return AssetVec,DepositVec

A deflator is used to deflate the monetary amounts. This comes from the FED.

In [247]:
deflator = dict()

deflator[2012] = 1
deflator[2013] = 1.01
deflator[2014] = 1.03
deflator[2015] = 1.03
deflator[2016] = 1.05
deflator[2017] = 1.07
deflator[2018] = 1.09
deflator[2019] = 1.11


The function is run on the obtained sample of M&A transactions and the results are appended to the list.

In [248]:
x,y = getMAAssetsAndDeposits(MATrans1417,deflator)

MATrans1417["Total Assets Prev Year"] = x
MATrans1417["Total Deposits Prev Year"] = y

In [249]:
MATrans1417Sorted = MATrans1417.sort_values("Total Assets Prev Year",ascending = False).dropna().reset_index(drop = True) # M&A Transactions are sorted by Total Assets Previous Year

This sample contains firms that have participated in multiple M&A transactions ina  +- 2 year window. These are excluded by using the following function to get the good indices:

In [250]:
def getTopXIndices(N,MATrans1417Sorted,MATrans1219):
    '''This Function returns the top N M&A Transactions by excluding companies that did multiple deals'''
    nGoodMA = 0
    i = 0

    # List of Good indices
    iGood = []


    while nGoodMA < N:
        # Obtain Acquirer and Target ID
        AcqID = MATrans1417Sorted["ACQ_CERT"][i]
        TarID = MATrans1417Sorted["OUT_CERT"][i]
        Year = MATrans1417Sorted["EFFYEAR"][i]

        # Create a copy of the extended DataFrame
        MATrans1219_ = MATrans1219[MATrans1219["EFFYEAR"]>= Year - 2]
        MATrans1219_ = MATrans1219_[MATrans1219_["EFFYEAR"]<= Year + 2]
        
        # Create list of all IDs in this window
        IDsWindow = np.hstack([MATrans1219_["ACQ_CERT"].to_numpy(),MATrans1219_["OUT_CERT"].to_numpy()])
        
        # Find positions of current Deal in this list
        OccurenceAcqID = np.where(IDsWindow == AcqID)[0]
        OccurenceTarID = np.where(IDsWindow == TarID)[0]

        # Check whether the a bank is part of more than one deal (we know it must be part of at least one)
        if OccurenceAcqID.shape[0] == 1 and OccurenceTarID.shape[0] == 1:

            nGoodMA += 1
            iGood.append(i)
        i += 1

    return iGood

In [251]:
indices = getTopXIndices(50,MATrans1417Sorted,MATrans1219) # Get Indices
MAPreCensor = MATrans1417Sorted.iloc[indices,:].reset_index(drop=True) # Get rid of all other M&A transactions


We get rid of two Deals: for i = 28 (First NBC Bank), there is no data in the whole observational period, and Southside Bank (i = 21) had two Deals (outside the windows), we exclude this to ensure a DMU is not compared to itself in the same time period.

In [252]:
# Rename Columns
MAPairs = MA[["ACQ_CERT","ACQ_INSTNAME","OUT_CERT","OUT_INSTNAME","EFFYEAR"]].rename(columns = {"ACQ_CERT":"ACQ","OUT_CERT":"TAR","EFFYEAR":"YEAR","ACQ_INSTNAME": "ACQ_Name","OUT_INSTNAME":"TAR_Name"})

# Drop two DMUs for reasons explained above
MAPairs.drop([28,21],inplace = True) # i = 28 closed before the end of the observation period

MAPairs = MAPairs.reset_index(drop=True)


The following Banks exhibit zeros in their data and need to be excluded. This observation was made at a later stage, but to ensure consistency of this notebok it is presented here:

In [253]:
BadDMU = [628,34775,21843]

In [254]:
MAPairs = MAPairs.loc[~MAPairs["ACQ"].isin(BadDMU)].reset_index(drop = True) # Drop Deals with zeros in Data

In [255]:
MAPairs

,ACQ,ACQ_Name,TAR,TAR_Name,YEAR
0,18409,"TD Bank, National Association",58656,Scottrade Bank,2017
1,588,Manufacturers and Traders Trust Company,13074,Hudson City Savings Bank,2015
2,58978,"CIT BANK, NATIONAL ASSOCIATION",35575,CIT Bank,2015
3,5510,Frost Bank,22482,Western National Bank,2014
4,12441,Hancock Bank,33029,Whitney Bank,2014
5,4988,Trustmark National Bank,34866,Reliance Bank,2017
6,26610,Bank of Hope,23301,Wilshire Bank,2016
7,15289,Great Western Bank,28092,Home Federal Bank,2016
8,17491,PlainsCapital Bank,32460,"Southwest Securities , FSB",2015
9,28178,Northwest Bank,14832,The Lorain National Bank,2015


Note that the names amy not match the ones in the Dissertation as they have been edited for clarity. One may check the FDIC bank finder for questions regarding the naming of the banks in the DIssertation.

## 2. Obtain and clean the data for the Banks in the sample

In this section the Financial Data for the Deals is presented. The following Excel Spreadsheets have been created manually by donwloading Data from the FDIC. Note that they may create extra observations that will be filtered out later.

In [256]:
IncomeExpenses = pd.read_excel("Data/IncomeAndExpenses.xlsx")
AssetLiability = pd.read_excel("Data/AssetsLiabilities.xlsx")

The following Variables are used:
- Inputs: Salaries & Emplyee Benefits (ESAL) AND Total Depsits (DEP)
- Outputs: Total Loans (ILNDOM + ILNFOR) AND Net Interest Income (NIM)
- Carry-Over (fixed): Bank Premises and Fixed Assets (BKPREM)

In [257]:
# Approprate columns are loaded
IncomeExpensesSubset = IncomeExpenses[["RISDATE","CERT","ESAL","ILNDOM","ILNFOR","NIM"]].copy()

AssetLiabilitySubset = AssetLiability[["RISDATE","CERT","DEP","BKPREM"]].copy()

IncomeExpensesYearList = []
AssetLiabilityYearList = []

# For each of the observations the Year is obtained by splitting the string into the first four digits and the rest
for i in range(IncomeExpensesSubset.shape[0]):
    IncomeExpensesYear = str(IncomeExpensesSubset["RISDATE"][i])
    AssetLiabilityYear = str(AssetLiabilitySubset["RISDATE"][i])

    IncomeExpensesYear = int(IncomeExpensesYear[0:4])
    AssetLiabilityYear = int(AssetLiabilityYear[0:4])

    IncomeExpensesYearList.append(IncomeExpensesYear)
    AssetLiabilityYearList.append(AssetLiabilityYear)


IncomeExpensesSubset.loc[:,"YEAR"] = IncomeExpensesYearList
AssetLiabilitySubset.loc[:,"YEAR"] = AssetLiabilityYearList


Most banks do not have an ILNFOR, this is filled with 0

In [258]:
IncomeExpensesSubset = IncomeExpensesSubset.fillna(0)

A new Pandas DataFrame is created with the results for the Financial Statements.

In [259]:
FinancialStatementMerged = pd.DataFrame(columns = ["Year","ID","ESAL","ILNDOM","ILNFOR","NIM","DEP","BKPREM"])

for i in range(IncomeExpensesSubset.shape[0]):

    # For each of the observation the Years and IDs are collected
    IncomeExpensesYear = IncomeExpensesSubset["YEAR"][i]
    AssetLiabilityYear = AssetLiabilitySubset["YEAR"][i]
    IncomeExpensesID = IncomeExpensesSubset["CERT"][i]
    AssetLiabilityID = AssetLiabilitySubset["CERT"][i]

    # A check is made if the Years/IDs math between the tables
    if IncomeExpensesYear != AssetLiabilityYear or IncomeExpensesID != AssetLiabilityID:
        raise ValueError("Something is not right")
    
    # The Values are collected
    ESAL = IncomeExpensesSubset["ESAL"][i]
    ILNDOM = IncomeExpensesSubset["ILNDOM"][i]
    ILNFOR = IncomeExpensesSubset["ILNFOR"][i]
    NIM = IncomeExpensesSubset["NIM"][i]
    DEP = AssetLiabilitySubset["DEP"][i]
    BKPREM = AssetLiabilitySubset["BKPREM"][i]
    
    # The Values are added to the DataFrame
    FinancialStatementMerged.loc[i,:] = [IncomeExpensesYear,IncomeExpensesID,ESAL,ILNDOM,ILNFOR,NIM,DEP,BKPREM]

Note that at this point FinancialStatementsMerged does not contain valeus for Acquired banks past year 2 (they merge in year 3 and cease to exist). This is changed in the following to ensure the matrices can be created easily later. Furthermore, the Data is deflated.

In [260]:
FinancialStatementMergedExtended = pd.DataFrame(columns = ["Year","ID","ESAL","ILNDOM","ILNFOR","NIM","DEP","BKPREM"])

j = 0

for i in range(MAPairs.shape[0]):

    # Get ID
    ACQID = MAPairs["ACQ"][i]
    TARID = MAPairs["TAR"][i]
    Year = MAPairs["YEAR"][i]

    # Add Acquirer
    AcqFinancialStatement = FinancialStatementMerged[FinancialStatementMerged["ID"] == ACQID].reset_index(drop = True)
    AcqFinancialStatementMAandLater = AcqFinancialStatement[AcqFinancialStatement["Year"]>=Year]

    # Add Target 
    TarFinancialStatement = FinancialStatementMerged[FinancialStatementMerged["ID"] == TARID].reset_index(drop = True)

    # Add Post MA rows
    TarFinancialStatement = pd.concat([AcqFinancialStatementMAandLater,TarFinancialStatement]).reset_index(drop = True)
    TarFinancialStatement["ID"] = TARID
    
    # Add the rows
    for k in range(5):
        
        FinancialStatementMergedExtended.loc[j,:] = AcqFinancialStatement.loc[k,:]
        FinancialStatementMergedExtended.iloc[j,2:] = FinancialStatementMergedExtended.iloc[j,2:]/deflator[Year]
        j += 1

    for k in range(5):
        
        FinancialStatementMergedExtended.loc[j,:] = TarFinancialStatement.loc[k,:]
        FinancialStatementMergedExtended.iloc[j,2:] = FinancialStatementMergedExtended.iloc[j,2:]/deflator[Year]
        j += 1
    

FinancialStatementMergedExtended now has 450 (90 Banks times 5 years) rows for each of the osbervations. At this point the column names are still according to the labels by the FDIC. This is changed in the following step:

In [261]:
InputOutputCarryPandas = pd.DataFrame(columns = ["Time","DMU","Salary","Deposits","Loans","InterestIncome","FixedAssets"])

j = 0

for i in range(45):

    # Target and Acquirer ID
    ACQID = MAPairs["ACQ"][i]
    TARID = MAPairs["TAR"][i]
    Year = MAPairs["YEAR"][i]
    
    # Data from the FinancialStatements is obtained
    FinancialStatementMergedExtendedACQ = FinancialStatementMergedExtended[FinancialStatementMergedExtended["ID"] == ACQID]
    FinancialStatementMergedExtendedTAR = FinancialStatementMergedExtended[FinancialStatementMergedExtended["ID"] == TARID]

    # The Current Year is obtained
    YearACQ = FinancialStatementMergedExtendedACQ["Year"].to_numpy()
    YearTAR = FinancialStatementMergedExtendedTAR["Year"].to_numpy()

    # The Salary is obtained
    SalaryACQ = FinancialStatementMergedExtendedACQ["ESAL"].to_numpy()
    SalaryTAR = FinancialStatementMergedExtendedTAR["ESAL"].to_numpy()

    # The Deposits are obtained
    DepositsACQ =  FinancialStatementMergedExtendedACQ["DEP"].to_numpy()
    DepositsTAR =  FinancialStatementMergedExtendedTAR["DEP"].to_numpy()

    # The Loans are obtained
    LoansACQ = (FinancialStatementMergedExtendedACQ["ILNDOM"] + FinancialStatementMergedExtendedACQ["ILNFOR"]).to_numpy()
    LoansTAR = (FinancialStatementMergedExtendedTAR["ILNDOM"] + FinancialStatementMergedExtendedTAR["ILNFOR"]).to_numpy()

    # The Interest Income is obtained
    InterestIncomeACQ = FinancialStatementMergedExtendedACQ["NIM"].to_numpy()
    InterestIncomeTAR = FinancialStatementMergedExtendedTAR["NIM"].to_numpy()

    # Fixed Assets are obtained
    FixedAssetsACQ = FinancialStatementMergedExtendedACQ["BKPREM"].to_numpy()
    FixedAssetsTAR = FinancialStatementMergedExtendedTAR["BKPREM"].to_numpy()

    for k in range(5):
        InputOutputCarryPandas.loc[j,:] = [4-k,ACQID,SalaryACQ[k],DepositsACQ[k],LoansACQ[k],InterestIncomeACQ[k],FixedAssetsACQ[k]]
        j += 1

    for k in range(5):
        InputOutputCarryPandas.loc[j,:] = [4-k,TARID,SalaryTAR[k],DepositsTAR[k],LoansTAR[k],InterestIncomeTAR[k],FixedAssetsTAR[k]]
        j += 1

InputOutputCarryPandas is a DataFrame in Long Format for all the Data needed for the (D)SBM-Model.

In [262]:
InputOutputCarryPandas.head(10)

,Time,DMU,Salary,Deposits,Loans,InterestIncome,FixedAssets
0,4,18409,2218062.616822,255705616.82243,5768828.037383,6451202.803738,4151618.691589
1,3,18409,2235386.915888,242615331.775701,5148003.738318,5979420.560748,2416184.11215
2,2,18409,2144234.579439,230299901.869159,4603175.700935,5366563.551402,2517177.570093
3,1,18409,2052319.626168,213958614.953271,4045617.757009,4986068.224299,2482735.514019
4,0,18409,1987415.88785,194847616.82243,3627262.616822,4556110.280374,2458763.551402
5,4,58656,2218062.616822,255705616.82243,5768828.037383,6451202.803738,4151618.691589
6,3,58656,2235386.915888,242615331.775701,5148003.738318,5979420.560748,2416184.11215
7,2,58656,2144234.579439,230299901.869159,4603175.700935,5366563.551402,2517177.570093
8,1,58656,16879.439252,14633606.542056,83754.205607,226565.420561,49375.700935
9,0,58656,14545.794393,13605406.542056,68171.962617,212399.065421,51809.345794


## 3. Create the matrices for the DSBM and SBM Model
Finally the matrices need to be created.

In [263]:
DMUnum = 45*2 # The Number of DMus

For each Input/Output/Fixed Carry-Over an empty tensor (multidmensional array) is created. This is then populated with the data.

In [264]:
InputMatrix = np.zeros(shape = (2,DMUnum,5))    # Empty Array

ACQnum = 0
TARnum = 0

# Going through all DMUS
for n in range(DMUnum):
        # Checking if current DMU is Target or Acqurier
        if n % 2 == 0:
                # Obtain Data and add it
                DMUID = MAPairs["ACQ"].to_numpy()[ACQnum]
                ACQnum +=1
                CurrentDMU = InputOutputCarryPandas[InputOutputCarryPandas["DMU"] == DMUID].reset_index(drop = True)

        if n % 2 == 1: 
                # Obtain Data and add it
                DMUID = MAPairs["TAR"].to_numpy()[TARnum]
                TARnum +=1
                CurrentDMU = InputOutputCarryPandas[InputOutputCarryPandas["DMU"] == DMUID].reset_index(drop = True)

        for T in range(5):
                SalaryDMU = CurrentDMU["Salary"][4-T]
                InputMatrix[0,n,T] = SalaryDMU


ACQnum = 0
TARnum = 0

# Going through all DMUS
for n in range(DMUnum):
    # Checking if current DMU is Target or Acqurier
        if n % 2 == 0: 
                # Obtain Data and add it
                DMUID = MAPairs["ACQ"].to_numpy()[ACQnum]
                ACQnum +=1
                CurrentDMU = InputOutputCarryPandas[InputOutputCarryPandas["DMU"] == DMUID].reset_index(drop = True)

        if n % 2 == 1: 
                # Obtain Data and add it
                DMUID = MAPairs["TAR"].to_numpy()[TARnum]
                TARnum +=1

                CurrentDMU = InputOutputCarryPandas[InputOutputCarryPandas["DMU"] == DMUID].reset_index(drop = True)

        for T in range(5):
                DepositDMU = CurrentDMU["Deposits"][4-T]
                InputMatrix[1,n,T] = DepositDMU



In [265]:
OutputMatrix = np.zeros(shape = (2,DMUnum,5))   # Empty Array

ACQnum = 0
TARnum = 0

# Going through all DMUS
for n in range(DMUnum):
        # Checking if current DMU is Target or Acqurier
        if n % 2 == 0: 
                # Obtain Data and add it
                DMUID = MAPairs["ACQ"].to_numpy()[ACQnum]
                ACQnum +=1
                CurrentDMU = InputOutputCarryPandas[InputOutputCarryPandas["DMU"] == DMUID].reset_index(drop = True)

        if n % 2 == 1: 
                # Obtain Data and add it
                DMUID = MAPairs["TAR"].to_numpy()[TARnum]
                TARnum +=1
                CurrentDMU = InputOutputCarryPandas[InputOutputCarryPandas["DMU"] == DMUID].reset_index(drop = True)

        for T in range(5):
                LoansDMU = CurrentDMU["Loans"][4-T]
                OutputMatrix[0,n,T] = LoansDMU


ACQnum = 0
TARnum = 0

# Going through all DMUS
for n in range(DMUnum):
        # Checking if current DMU is Target or Acqurier
        if n % 2 == 0: 
                # Obtain Data and add it
                DMUID = MAPairs["ACQ"].to_numpy()[ACQnum]
                ACQnum +=1
                CurrentDMU = InputOutputCarryPandas[InputOutputCarryPandas["DMU"] == DMUID].reset_index(drop = True)

        if n % 2 == 1: 
                # Obtain Data and add it
                DMUID = MAPairs["TAR"].to_numpy()[TARnum]
                TARnum +=1
                CurrentDMU = InputOutputCarryPandas[InputOutputCarryPandas["DMU"] == DMUID].reset_index(drop = True)

        for T in range(5):
                InterestIncomeDMU = CurrentDMU["InterestIncome"][4-T]
                OutputMatrix[1,n,T] = InterestIncomeDMU

In [266]:
CarryOverMatrix = np.zeros(shape = (1,DMUnum,5))        # Empty Array

ACQnum = 0
TARnum = 0

# Going through all DMUS
for n in range(DMUnum):
        # Checking if current DMU is Target or Acqurier
        if n % 2 == 0: 
                # Obtain Data and add it
                DMUID = MAPairs["ACQ"].to_numpy()[ACQnum]
                ACQnum +=1
                CurrentDMU = InputOutputCarryPandas[InputOutputCarryPandas["DMU"] == DMUID].reset_index(drop = True)

        if n % 2 == 1:
                # Obtain Data and add it
                DMUID = MAPairs["TAR"].to_numpy()[TARnum]
                TARnum +=1
                CurrentDMU = InputOutputCarryPandas[InputOutputCarryPandas["DMU"] == DMUID].reset_index(drop = True)

        for T in range(5):
                FixedAssetsDMU = CurrentDMU["FixedAssets"][4-T]
                CarryOverMatrix[0,n,T] = FixedAssetsDMU

Finally for the DSBm-Model,the Matrices are reshaped and then saved:

In [267]:
InputMatrix_ = InputMatrix.reshape(2,DMUnum*5)
OutputMatrix_ = OutputMatrix.reshape(2,DMUnum*5)
CarryOverMatrix_ = CarryOverMatrix.reshape(DMUnum,5)

In [268]:
np.savetxt("Data/DSBMInputMatrix.txt",X = InputMatrix_)
np.savetxt("Data/DSBMOutputMatrix.txt",X = OutputMatrix_)
np.savetxt("Data/DSBMCarryOverMatrix.txt",X = CarryOverMatrix_)

For the SBM-Model the Data for the Year of the Merger (for the actual banks) and of the Year preceeding the Merger (for the virtual banks) is needed.

In [269]:
SBMDataMAYear = InputOutputCarryPandas[InputOutputCarryPandas["Time"] == 2]
SBMDataPreMAYear = InputOutputCarryPandas[InputOutputCarryPandas["Time"] == 1]

Those observations that correspond to IDs of Acquirers are obtained and filtered.

In [270]:
SBMDataMAYearFiltered = pd.DataFrame(columns = ["DMU","Salary","Deposits","FixedAssets","Loans","InterestIncome"]) # New DataFrame

# Run through all DMUs
for i in range(int(DMUnum/2)):

    # Get the data
    x = SBMDataMAYear[SBMDataMAYear["DMU"]==MAPairs["ACQ"][i]].copy()
    x.drop(columns = "Time",inplace = True)
    x = x.to_numpy().reshape(6)

    # Add the data
    SBMDataMAYearFiltered.loc[i] = x

Those observations that correspond to the data of the previous year for Targets and Acquirer are obtained and combined.

In [271]:
SBMDataPreMAYearFiltered = pd.DataFrame(columns = ["DMU","Salary","Deposits","FixedAssets","Loans","InterestIncome"])   # New Data Frame

# Run through all DMUs
for i in range(int(DMUnum/2)):

    # Get the data for the Acquirer
    x = SBMDataPreMAYear[SBMDataPreMAYear["DMU"]==MAPairs["ACQ"][i]].copy()
    x.drop(columns = "Time",inplace = True)
    x = x.to_numpy().reshape(6)
    name = x[0]
    name = f"Virtual{name}"
    
    # Get the data for the Target
    y = SBMDataPreMAYear[SBMDataPreMAYear["DMU"]==MAPairs["TAR"][i]].copy()
    y.drop(columns = "Time",inplace = True)
    y = y.to_numpy().reshape(6)

    # Add the data together and add it to the DataFrame
    xy = x+y
    xy[0] = name
    SBMDataPreMAYearFiltered.loc[i] = xy


Input and Output Matrices are created.

In [272]:
InputMatrixSBM = np.zeros((3,DMUnum))

# Create Indices to keep track of the array creation
i = 0
iMA = 0
iPreMA = 0
while i < 90:

    # Check wether DMU is Acquirer or Target
    if i % 2 == 0:

        # Add data
        InputMatrixSBM[0,i] = SBMDataMAYearFiltered["Salary"][iMA]
        InputMatrixSBM[1,i] = SBMDataMAYearFiltered["Deposits"][iMA]
        InputMatrixSBM[2,i] = SBMDataMAYearFiltered["FixedAssets"][iMA]

        iMA += 1


    if i % 2 == 1:

        # Add data
        InputMatrixSBM[0,i] = SBMDataPreMAYearFiltered["Salary"][iPreMA]
        InputMatrixSBM[1,i] = SBMDataPreMAYearFiltered["Deposits"][iPreMA]
        InputMatrixSBM[2,i] = SBMDataPreMAYearFiltered["FixedAssets"][iPreMA]

        iPreMA += 1

    i += 1
    


In [273]:
OutputMatrixSBM = np.zeros((2,DMUnum))

# Create Indices to keep track of the array creation
i = 0
iMA = 0
iPreMA = 0
while i < 90:

    # Check wether DMU is Acquirer or Target
    if i % 2 == 0:

        # Add data
        OutputMatrixSBM[0,i] = SBMDataMAYearFiltered["Loans"][iMA]
        OutputMatrixSBM[1,i] = SBMDataMAYearFiltered["InterestIncome"][iMA]

        iMA += 1


    if i % 2 == 1:

        # Add data
        OutputMatrixSBM[0,i] = SBMDataPreMAYearFiltered["Loans"][iPreMA]
        OutputMatrixSBM[1,i] = SBMDataPreMAYearFiltered["InterestIncome"][iPreMA]

        iPreMA += 1

    i += 1
    


Finally for the SBM-Model the matrices are saved to be used later.

In [274]:
np.savetxt("Data/SBMInputMatrix.txt",X = InputMatrixSBM)
np.savetxt("Data/SBMOutputMatrix.txt",X = OutputMatrixSBM)